In [1]:
import pandas as pd
import pymongo
import numpy as np
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func


In [2]:
firedata_file = "templates/filtered_fire.csv"
fire_df = pd.read_csv(firedata_file)
fire_df.head()

,Unnamed: 0,X,Y,OBJECTID,GLOBALID,FIREOCCURID,CN,REVDATE,FIRENAME,COMPLEXNAME,...,PROTECTIONAGENCY,UNITIDPROTECT,LATDD83,LONGDD83,FIRETYPECATEGORY,POINTTYPE,PERIMEXISTS,FIRESTATQC,DBSOURCEID,DBSOURCEDATE
0,765,-114.939788,47.218545,329408702,NaN,NaN,1518600,1999/11/30 00:00:00+00,WestMullan,NaN,...,NaN,NaN,47.21855,-114.93979,WF,,,,1,2020/09/11 04:04:00+00
1,1306,-110.439444,45.046111,329409243,NaN,NaN,1500990,1999/11/30 00:00:00+00,Bull,NaN,...,NaN,NaN,45.04611,-110.43944,WF,,,,1,2020/09/11 04:04:03+00
2,2676,-114.880556,46.448889,329410613,NaN,NaN,1507394,1999/11/30 00:00:00+00,Queen,NaN,...,NaN,NaN,46.44889,-114.88056,WF,,,,1,2020/09/11 04:04:10+00
3,2677,-114.939444,46.399444,329410614,NaN,NaN,1507396,1999/11/30 00:00:00+00,FreezeOut,NaN,...,NaN,NaN,46.39944,-114.93944,WF,,,,1,2020/09/11 04:04:10+00
4,2967,-114.351944,46.746667,329410904,NaN,NaN,1517199,1999/11/30 00:00:00+00,WestFork2,NaN,...,NaN,NaN,46.74667,-114.35194,WF,,,,1,2020/09/11 04:04:11+00


In [3]:
filtered_fire_df = fire_df[fire_df['FIREYEAR'] >=2010.0]
filtered_fire_df




,Unnamed: 0,X,Y,OBJECTID,GLOBALID,FIREOCCURID,CN,REVDATE,FIRENAME,COMPLEXNAME,...,PROTECTIONAGENCY,UNITIDPROTECT,LATDD83,LONGDD83,FIRETYPECATEGORY,POINTTYPE,PERIMEXISTS,FIRESTATQC,DBSOURCEID,DBSOURCEDATE
0,765,-114.939788,47.218545,329408702,NaN,NaN,1518600,1999/11/30 00:00:00+00,WestMullan,NaN,...,NaN,NaN,47.21855,-114.93979,WF,,,,1,2020/09/11 04:04:00+00
1,1306,-110.439444,45.046111,329409243,NaN,NaN,1500990,1999/11/30 00:00:00+00,Bull,NaN,...,NaN,NaN,45.04611,-110.43944,WF,,,,1,2020/09/11 04:04:03+00
2,2676,-114.880556,46.448889,329410613,NaN,NaN,1507394,1999/11/30 00:00:00+00,Queen,NaN,...,NaN,NaN,46.44889,-114.88056,WF,,,,1,2020/09/11 04:04:10+00
3,2677,-114.939444,46.399444,329410614,NaN,NaN,1507396,1999/11/30 00:00:00+00,FreezeOut,NaN,...,NaN,NaN,46.39944,-114.93944,WF,,,,1,2020/09/11 04:04:10+00
4,2967,-114.351944,46.746667,329410904,NaN,NaN,1517199,1999/11/30 00:00:00+00,WestFork2,NaN,...,NaN,NaN,46.74667,-114.35194,WF,,,,1,2020/09/11 04:04:11+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1765,552412,-92.197778,36.714167,329960349,{9B75D7B5-16AC-4600-BA5B-3B4F2FF59905},NaN,1628803,1999/11/30 00:00:00+00,Potato Cave,NaN,...,USFS,MOMTF,36.71417,-92.19778,WF,Unknown,Y,No,9,2020/09/11 05:03:31+00
1766,552707,-93.050556,36.818611,329960644,{11D8D179-9042-487E-8FF5-4018A98BF724},NaN,1701441,1999/11/30 00:00:00+00,Deveraux,NaN,...,USA,MOMTF,36.81861,-93.05056,WF,Unknown,Y,No,9,2020/09/11 05:03:32+00
1767,552741,-91.336944,37.472222,329960678,{24220C03-9A10-4F2B-896D-DC9096D22241},NaN,1611751,1999/11/30 00:00:00+00,Minninghaw,NaN,...,USA,MOMTF,37.47222,-91.33694,WF,Unknown,Y,No,9,2020/09/11 05:03:32+00
1768,553110,-93.069444,36.848333,329961047,{584EC429-4D48-4411-9B9D-40794CD91F3B},NaN,1567317,1999/11/30 00:00:00+00,Deveraux,NaN,...,USFS,MOMTF,36.84833,-93.06944,WF,Unknown,Y,No,9,2020/09/11 05:03:35+00


In [4]:
filtered_fire_df1 = filtered_fire_df[filtered_fire_df['TOTALACRES'] > 500]
filtered_fire_df1

,Unnamed: 0,X,Y,OBJECTID,GLOBALID,FIREOCCURID,CN,REVDATE,FIRENAME,COMPLEXNAME,...,PROTECTIONAGENCY,UNITIDPROTECT,LATDD83,LONGDD83,FIRETYPECATEGORY,POINTTYPE,PERIMEXISTS,FIRESTATQC,DBSOURCEID,DBSOURCEDATE
0,765,-114.939788,47.218545,329408702,NaN,NaN,1518600,1999/11/30 00:00:00+00,WestMullan,NaN,...,NaN,NaN,47.21855,-114.93979,WF,,,,1,2020/09/11 04:04:00+00
1,1306,-110.439444,45.046111,329409243,NaN,NaN,1500990,1999/11/30 00:00:00+00,Bull,NaN,...,NaN,NaN,45.04611,-110.43944,WF,,,,1,2020/09/11 04:04:03+00
2,2676,-114.880556,46.448889,329410613,NaN,NaN,1507394,1999/11/30 00:00:00+00,Queen,NaN,...,NaN,NaN,46.44889,-114.88056,WF,,,,1,2020/09/11 04:04:10+00
3,2677,-114.939444,46.399444,329410614,NaN,NaN,1507396,1999/11/30 00:00:00+00,FreezeOut,NaN,...,NaN,NaN,46.39944,-114.93944,WF,,,,1,2020/09/11 04:04:10+00
4,2967,-114.351944,46.746667,329410904,NaN,NaN,1517199,1999/11/30 00:00:00+00,WestFork2,NaN,...,NaN,NaN,46.74667,-114.35194,WF,,,,1,2020/09/11 04:04:11+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1765,552412,-92.197778,36.714167,329960349,{9B75D7B5-16AC-4600-BA5B-3B4F2FF59905},NaN,1628803,1999/11/30 00:00:00+00,Potato Cave,NaN,...,USFS,MOMTF,36.71417,-92.19778,WF,Unknown,Y,No,9,2020/09/11 05:03:31+00
1766,552707,-93.050556,36.818611,329960644,{11D8D179-9042-487E-8FF5-4018A98BF724},NaN,1701441,1999/11/30 00:00:00+00,Deveraux,NaN,...,USA,MOMTF,36.81861,-93.05056,WF,Unknown,Y,No,9,2020/09/11 05:03:32+00
1767,552741,-91.336944,37.472222,329960678,{24220C03-9A10-4F2B-896D-DC9096D22241},NaN,1611751,1999/11/30 00:00:00+00,Minninghaw,NaN,...,USA,MOMTF,37.47222,-91.33694,WF,Unknown,Y,No,9,2020/09/11 05:03:32+00
1768,553110,-93.069444,36.848333,329961047,{584EC429-4D48-4411-9B9D-40794CD91F3B},NaN,1567317,1999/11/30 00:00:00+00,Deveraux,NaN,...,USFS,MOMTF,36.84833,-93.06944,WF,Unknown,Y,No,9,2020/09/11 05:03:35+00


In [5]:
filtered_fire_df1['STATCAUSE'].unique()

array(['9 - Miscellaneous', '1 - Lightning', '7 - Arson',
       '2 - Equipment Use', '4 - Campfire', '5 - Debris Burning',
       '0 - Unknown', '6 - Railroad', '3 - Smoking', '8 - Children'],
      dtype=object)

In [6]:
filtered_fire_df1['STATCAUSE'] = filtered_fire_df1['STATCAUSE'].replace({1.0 :'1 - Lightning',
                                                                         '1': '1 - Lightning',
                                                                         '1 -  Lightning': '1 - Lightning',
                                                                         'Lightning':'1 - Lightning',
                                                                         2.0 :'2 - Equipment Use',
                                                                         '2 -  Equipment Use':'2 - Equipment Use',
                                                                         '2':'2 - Equipment Use',
                                                                         0 : '0 - Unknown',
                                                                         ' -  ':'0 - Unknown',
                                                                         np.nan:'0 - Unknown',
                                                                         3.0:'3 - Smoking',
                                                                         4.0:'4 - Campfire',
                                                                         'Campfire':'4 - Campfire',
                                                                         '4 -  Campfire':'4 - Campfire',
                                                                         '4':'4 - Campfire',
                                                                         5.0:'5 - Debris Burning',
                                                                         '5 -  Debris Burning':'5 - Debris Burning',
                                                                         '5':'5 - Debris Burning',
                                                                         'Debris Burning':'5 - Debris Burning',
                                                                         6.0:'6 - Railroad',
                                                                         '6':'6 - Railroad',
                                                                         7.0:'7 - Arson',
                                                                         'Arson':'7 - Arson',
                                                                         '7 -  Arson':'7 - Arson',
                                                                         '7':'7 - Arson',
                                                                         8:'8 - Children',
                                                                         9.0:'9 - Miscellaneous',
                                                                         '9 -  Miscellaneous':'9 - Miscellaneous',
                                                                         '9':'9 - Miscellaneous',
                                                                         'Miscellaneous':'9 - Miscellaneous',
                                                                         11:'9 - Miscellaneous',
                                                                         '11':'9 - Miscellaneous',
                                                                         '12':'9 - Miscellaneous',
                                                                         '13':'9 - Miscellaneous',
                                                                         13:'9 - Miscellaneous'})

# df.Num_of_employees = df.Num_of_employees.replace({"10-Jan": "1-10",
#                                                    "Nov-50": "11-50"})


In [7]:
filtered_fire_df1.groupby(['STATCAUSE']).count()



,Unnamed: 0,X,Y,OBJECTID,GLOBALID,FIREOCCURID,CN,REVDATE,FIRENAME,COMPLEXNAME,...,PROTECTIONAGENCY,UNITIDPROTECT,LATDD83,LONGDD83,FIRETYPECATEGORY,POINTTYPE,PERIMEXISTS,FIRESTATQC,DBSOURCEID,DBSOURCEDATE
STATCAUSE,,,,,,,,,,,,,,,,,,,,,
0 - Unknown,18,18,18,18,8,6,7,18,18,0,...,12,9,18,18,18,18,18,12,18,18
1 - Lightning,1091,1091,1091,1091,606,522,775,1089,1091,129,...,753,611,1091,1091,1091,1091,1091,1078,1091,1091
2 - Equipment Use,85,85,85,85,35,20,25,85,84,6,...,48,42,85,85,85,85,85,82,85,85
3 - Smoking,2,2,2,2,1,0,1,2,2,1,...,2,2,2,2,2,2,2,2,2,2
4 - Campfire,57,57,57,57,34,22,32,57,57,1,...,37,28,57,57,57,57,57,57,57,57
5 - Debris Burning,15,15,15,15,6,2,7,15,15,0,...,8,7,15,15,15,15,15,13,15,15
6 - Railroad,11,11,11,11,7,2,7,11,11,5,...,7,5,11,11,11,11,11,11,11,11
7 - Arson,94,94,94,94,51,42,51,94,94,12,...,77,64,94,94,94,94,94,92,94,94
8 - Children,3,3,3,3,1,1,1,3,3,1,...,1,1,3,3,3,3,3,3,3,3


In [8]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

db = client.fire_db

In [9]:
db.filtered_fire.insert_many(filtered_fire_df1.to_dict('records'))

In [10]:
filtered_fire_df1.to_csv('filtered_fire.csv', sep=',', encoding='utf-8')